In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import dataset_handler as dh
import testing_utils as tu
from termcolor import colored
import re
import numpy as np

/home/CodeGen_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
priming_text_path = "data/priming_texts/gsm8k/codegen/func_eq_short.txt"
gsm8k_path = "data/grade-school-math/grade_school_math/data/train.jsonl"

current_dataset = dh.init_dataset_from_name(
        "gsm8k", primingtext_path=priming_text_path
    )

tu.set_all_seeds()

rand_indexes = np.random.randint(0, len(current_dataset.data), 100)
print(rand_indexes)

test_data = [current_dataset.data[i] for i in rand_indexes]
test_data = [elem["question"] for elem in test_data]

In [5]:
test_data[0]

'Carol sells tickets for an exhibition. During three days she sold tickets worth $960. One ticket costs $4. How many tickets on average did she sell during one of these three days?'

In [6]:
import pickle

#pickle loads the model
clustering_model = pickle.load(open("clustering_model_3.pkl", "rb"))

In [7]:
embedder = SentenceTransformer('all-mpnet-base-v2')

# Corpus with example sentences
corpus = test_data
corpus_embeddings = embedder.encode(corpus)

In [8]:
labels = list(clustering_model.predict(corpus_embeddings))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
pkl_dir = "clustering_exp/3_cluster/"

with open(f"{pkl_dir}cluster_0_pass_at_k.pkl", "rb") as f:
    cluster_0_pass_at_k = pickle.load(f)
with open(f"{pkl_dir}cluster_1_pass_at_k.pkl", "rb") as f:
    cluster_1_pass_at_k = pickle.load(f)
with open(f"{pkl_dir}cluster_2_pass_at_k.pkl", "rb") as f:
    cluster_2_pass_at_k = pickle.load(f)
'''
with open(f"{pkl_dir}cluster_3_pass_at_k.pkl", "rb") as f:
    cluster_3_pass_at_k = pickle.load(f)'''

'\nwith open(f"{pkl_dir}cluster_3_pass_at_k.pkl", "rb") as f:\n    cluster_3_pass_at_k = pickle.load(f)'

In [12]:
agg_pass_at_k = []
max_dict = {}
max_dict[0] = []
max_dict[1] = []
max_dict[2] = []
#max_dict[3] = []


for i, label in enumerate(labels):
    cand_list = [
        cluster_0_pass_at_k[i],
        cluster_1_pass_at_k[i],
        cluster_2_pass_at_k[i],
        #cluster_3_pass_at_k[i],
    ]
    max_cand = cand_list[label]
    max_dict[cand_list.index(max_cand)].append(i)
    agg_pass_at_k.append(max_cand)

In [14]:
print(f"\nAggregate prompts - Pass@{3} = {np.mean(np.array(agg_pass_at_k))}\n")
print(f"cluster_0: {len(max_dict[0])}")
print(f"cluster_1: {len(max_dict[1])}")
print(f"cluster_2: {len(max_dict[2])}")
#print(f"cluster_3: {len(max_dict[3])}")


Aggregate prompts - Pass@3 = 0.38300000000000006

cluster_0: 75
cluster_1: 20
cluster_2: 5


In [23]:
with open(f"{pkl_dir}general_pass_at_k.pkl", "rb") as f:
    general_pass_at_k = pickle.load(f)

print(f"General prompts - Pass@{3} = {np.mean(np.array(general_pass_at_k))}")

General prompts - Pass@3 = 0.355


In [27]:
len(list(np.nonzero(np.array(agg_pass_at_k)))[0])

46